# Lab Assignment 9: Data Management Using `pandas`, Part 2
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, we are going to build the Country Analysis Relational DataBase (which we will call the C.A.R.D.B. or the "Cardi B"):

![CardbiB](https://media.giphy.com/media/3oEjI5ry4IwZ3RDw6k/giphy.gif "cardib")

We will be collecting data from two sources. First, we will use open data from the World Bank's [Sovereign
Environmental, Social, and Governance (ESG) Data](https://datatopics.worldbank.org/esg/) project. The ESG data reports data from every country in the world over the time frame from 1960-2022 on a wide variety of topics including education, health, and economic factors within the countries. Second, we will use data on the quality and democratic character of countries' governments as reported by the [Varieties of Democracy (V-Dem)](https://www.v-dem.net/data/the-v-dem-dataset/) project at the University of Notre Dame. By using both data sources, we can conduct analyses to see whether democratic openness leads to better societal outcomes for countries. We can also write queries to capture a wide range of information on countries' political parties, tax systems, and banking industries, for example. Or as Cardi B would say, "You in the club just to party, I'm there, I get paid a fee. I be in and out them banks so much, I know they're tired of me."

## Problem 0
Import the following packages (use `pip install` to download any packages you don't already have installed):

In [1]:
import numpy as np
import pandas as pd
import requests
import os
import io
import zipfile

Both the World Bank and V-Dem store their data in zipped directories containing CSV files. Download the World Bank data into your current working directory by typing the following code:

In [2]:
url = 'https://databank.worldbank.org/data/download/ESG_CSV.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

And download the V-Dem data by typing:

In [3]:
url = 'https://v-dem.net/media/datasets/V-Dem-CY-Core_csv_v13.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

After you've run this code successfully once, the files you need will be in your working directory and you should save time by switching these cells from "code" to "raw" so that they don't run again if you restart the kernel.

You will only need three of the files you've downloaded. Load the 'V-Dem-CY-Core-v13.csv' file as `vdem` and the 'ESGData.csv' file as `wb`.

In [4]:
vdem = pd.read_csv('V-Dem-CY-Core-v13.csv')
wb = pd.read_csv('ESGData.csv')

## Problem 1
First, let's focus on the `vdem` data ('V-Dem-CY-Core-v13.csv'). Use `pandas` methods to perform the following tasks:

### Part a
Keep only the 'country_text_id', 'country_name','year', 'v2x_polyarchy', and 'v2peedueq' columns. [1 point]

In [5]:
vdem = vdem[['country_text_id', 'country_name', 'year', 'v2x_polyarchy', 'v2peedueq']]
vdem

### Part b
Use the `.query()` method to keep only the rows in which year is greater than or equal to 1960 and less than or equal to 2021. [1 point]

In [6]:
vdem = vdem.query('year >= 1960 and year <= 2021')
vdem

### Part c
Rename 'country_text_id' to 'country_code', 'country_name' to 'country_name_vdem', 'v2x_polyarchy' to 'democracy', and 'v2peedueq' to 'educational_equality'. [1 point]

In [9]:
vdem = vdem.rename(columns={'country_text_id': 'country_code', 'country_name': 'country_name_vdem',
                            'v2x_polyarchy': 'democracy', 'v2peedueq': 'educational_equality'})
vdem

,country_code,country_name_vdem,year,democracy,educational_equality
171,MEX,Mexico,1960,0.232,-1.438
172,MEX,Mexico,1961,0.234,-1.438
173,MEX,Mexico,1962,0.233,-1.438
174,MEX,Mexico,1963,0.233,-1.438
175,MEX,Mexico,1964,0.231,-1.438
...,...,...,...,...,...
26150,ZZB,Zanzibar,2017,0.267,1.661
26151,ZZB,Zanzibar,2018,0.268,1.486
26152,ZZB,Zanzibar,2019,0.266,1.486
26153,ZZB,Zanzibar,2020,0.258,1.427


### Part d
Sort the rows by 'country_code' and 'year' in ascending order. [1 point]

In [8]:
vdem = vdem.sort_values(by=['country_code', 'year'], ascending=[True, True])
vdem

## Problem 2
Next focus on the World Bank `wb` dataset 'ESGData.csv'. Use `pandas` methods to perform the following tasks:

### Part a
Keep only the columns named 'Country Code', 'Country Name', and 'Indicator Code', or begin with '19' or '20'. (Don't type in all the years individually. Instead, use code that finds all columns that begin '19' or '20'.) [1 point]

In [ ]:
wb = wb[['Country Code', 'Country Name', 'Indicator Code'] + [col for col in wb.columns if col.startswith(('19', '20'))]]


### Part b
Rename 'Country Code' to'country_code', 'Country Name' to 'country_name_wb', and 'Indicator Code' to 'feature'. [1 point]

In [ ]:
wb = wb.rename(columns={'Country Code': 'country_code', 'Country Name': 'country_name_wb', 'Indicator Code': 'feature'})


### Part c
Use the `.query()` method to remove the rows in which 'country_name_wb' is equal to one of the entries in the folowing `noncountries` list: [1 point]

In [ ]:
noncountries = ["Arab World", "Central Europe and the Baltics",
                "Caribbean small states",
                "East Asia & Pacific (excluding high income)",
                "Early-demographic dividend","East Asia & Pacific",
                "Europe & Central Asia (excluding high income)",
                "Europe & Central Asia", "Euro area",
                "European Union","Fragile and conflict affected situations",
                "High income",
                "Heavily indebted poor countries (HIPC)","IBRD only",
                "IDA & IBRD total",
                "IDA total","IDA blend","IDA only",
                "Latin America & Caribbean (excluding high income)",
                "Latin America & Caribbean",
                "Least developed countries: UN classification",
                "Low income","Lower middle income","Low & middle income",
                "Late-demographic dividend","Middle East & North Africa",
                "Middle income",
                "Middle East & North Africa (excluding high income)",
                "North America","OECD members",
                "Other small states","Pre-demographic dividend",
                "Pacific island small states",
                "Post-demographic dividend",
                "Sub-Saharan Africa (excluding high income)",
                "Sub-Saharan Africa",
                "Small states","East Asia & Pacific (IDA & IBRD)",
                "Europe & Central Asia (IDA & IBRD)",
                "Latin America & Caribbean (IDA & IBRD)",
                "Middle East & North Africa (IDA & IBRD)","South Asia",
                "South Asia (IDA & IBRD)",
                "Sub-Saharan Africa (IDA & IBRD)",
                "Upper middle income", "World"]

In [ ]:
wb = wb.query('country_name_wb not in @noncountries')


### Part d
The features in this dataset are given strange and incomprehensible codes such as 'EG.CFT.ACCS.ZS'. Use the `replace_map` dictionary, defined below, to recode all of these values with more descriptive names for each feature. [1 point]

In [ ]:
replace_map = {
  "AG.LND.AGRI.ZS": "agricultural_land",
  "AG.LND.FRST.ZS": "forest_area",
  "AG.PRD.FOOD.XD": "food_production_index",
  "CC.EST": "control_of_corruption",
  "EG.CFT.ACCS.ZS": "access_to_clean_fuels_and_technologies_for_cooking",
  "EG.EGY.PRIM.PP.KD": "energy_intensity_level_of_primary_energy",
  "EG.ELC.ACCS.ZS": "access_to_electricity",
  "EG.ELC.COAL.ZS": "electricity_production_from_coal_sources",
  "EG.ELC.RNEW.ZS": "renewable_electricity_output",
  "EG.FEC.RNEW.ZS": "renewable_energy_consumption",
  "EG.IMP.CONS.ZS": "energy_imports",
  "EG.USE.COMM.FO.ZS": "fossil_fuel_energy_consumption",
  "EG.USE.PCAP.KG.OE": "energy_use",
  "EN.ATM.CO2E.PC": "co2_emissions",
  "EN.ATM.METH.PC": "methane_emissions",
  "EN.ATM.NOXE.PC": "nitrous_oxide_emissions",
  "EN.ATM.PM25.MC.M3": "pm2_5_air_pollution",
  "EN.CLC.CDDY.XD": "cooling_degree_days",
  "EN.CLC.GHGR.MT.CE": "ghg_net_emissions",
  "EN.CLC.HEAT.XD": "heat_index_35",
  "EN.CLC.MDAT.ZS": "droughts",
  "EN.CLC.PRCP.XD": "maximum_5-day_rainfall",
  "EN.CLC.SPEI.XD": "mean_drought_index",
  "EN.MAM.THRD.NO": "mammal_species",
  "EN.POP.DNST": "population_density",
  "ER.H2O.FWTL.ZS": "annual_freshwater_withdrawals",
  "ER.PTD.TOTL.ZS": "terrestrial_and_marine_protected_areas",
  "GB.XPD.RSDV.GD.ZS": "research_and_development_expenditure",
  "GE.EST": "government_effectiveness",
  "IC.BUS.EASE.XQ": "ease_of_doing_business_rank",
  "IC.LGL.CRED.XQ": "strength_of_legal_rights_index",
  "IP.JRN.ARTC.SC": "scientific_and_technical_journal_articles",
  "IP.PAT.RESD": "patent_applications",
  "IT.NET.USER.ZS": "individuals_using_the_internet",
  "NV.AGR.TOTL.ZS": "agriculture",
  "NY.ADJ.DFOR.GN.ZS": "net_forest_depletion",
  "NY.ADJ.DRES.GN.ZS": "natural_resources_depletion",
  "NY.GDP.MKTP.KD.ZG": "gdp_growth",
  "PV.EST": "political_stability_and_absence_of_violence",
  "RL.EST": "rule_of_law",
  "RQ.EST": "regulatory_quality",
  "SE.ADT.LITR.ZS": "literacy_rate",
  "SE.ENR.PRSC.FM.ZS": "gross_school_enrollment",
  "SE.PRM.ENRR": "primary_school_enrollment",
  "SE.XPD.TOTL.GB.ZS": "government_expenditure_on_education",
  "SG.GEN.PARL.ZS": "proportion_of_seats_held_by_women_in_national_parliaments",
  "SH.DTH.COMM.ZS": "cause_of_death",
  "SH.DYN.MORT": "mortality_rate",
  "SH.H2O.SMDW.ZS": "people_using_safely_managed_drinking_water_services",
  "SH.MED.BEDS.ZS": "hospital_beds",
  "SH.STA.OWAD.ZS": "prevalence_of_overweight",
  "SH.STA.SMSS.ZS": "people_using_safely_managed_sanitation_services",
  "SI.DST.FRST.20": "income_share_held_by_lowest_20pct",
  "SI.POV.GINI": "gini_index",
  "SI.POV.NAHC": "poverty_headcount_ratio_at_national_poverty_lines",
  "SI.SPR.PCAP.ZG": "annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income",
  "SL.TLF.0714.ZS": "children_in_employment",
  "SL.TLF.ACTI.ZS": "labor_force_participation_rate",
  "SL.TLF.CACT.FM.ZS": "ratio_of_female_to_male_labor_force_participation_rate",
  "SL.UEM.TOTL.ZS": "unemployment",
  "SM.POP.NETM": "net_migration",
  "SN.ITK.DEFC.ZS": "prevalence_of_undernourishment",
  "SP.DYN.LE00.IN": "life_expectancy_at_birth",
  "SP.DYN.TFRT.IN": "fertility_rate",
  "SP.POP.65UP.TO.ZS": "population_ages_65_and_above",
  "SP.UWT.TFRT": "unmet_need_for_contraception",
  "VA.EST": "voice_and_accountability",
  "EN.CLC.CSTP.ZS": "coastal_protection",
  "SD.ESR.PERF.XQ": "economic_and_social_rights_performance_score",
  "EN.CLC.HDDY.XD": "heating_degree_days",
  "EN.LND.LTMP.DC": "land_surface_temperature",
  "ER.H2O.FWST.ZS": "freshwater_withdrawal",
  "EN.H2O.BDYS.ZS": "water_quality",
  "AG.LND.FRLS.HA": "tree_cover_loss",
}


In [ ]:
wb['feature'] = wb['feature'].replace(replace_map)

## Problem 3
The `wb` dataset is strangely organized. The features are stored in the rows, when typically we would want these features to be columns. Also, years are stored in columns, when typically we would want years to be represented by different rows. We can repair this structure by reshaping the data.

### Part a
First, reshape the data to turn the columns that refer to years into rows. [1 point]

In [ ]:
wb = pd.melt(wb, id_vars=['country_code', 'country_name_wb', 'feature'], var_name='year')


### Part b
Then rename `variable` to `year`, and reshape the data again by turning the rows that refer to features into columns. [1 point]

In [ ]:
wb = pd.melt(wb, id_vars=['country_code', 'country_name_wb', 'feature'], var_name='year')


### Part c
After these reshapes, the year column in the `wb` data frame is stored as a string. Convert this column to an integer data type. [1 point]

In [ ]:
wb['year'] = wb['year'].astype(int)


## Problem 4
Next we will merge the `wb` data frame with the `vdem` data frame, matching on the 'country_code' and 'year' columns.

### Part a
First, write a sentence stating whether you expect this merge to be one-to-one, many-to-one, one-to-many, or many-to-many, and describe your rationale. [1 point]

I expect this merge to be many-to-many since one country can have multiple observations in both datasets.



### Part b
Next, merge the two datasets together in a way that checks whether your expectation is met, and also allows you to see the rows that failed to match. [2 points]

In [ ]:
merged = pd.merge(vdem, wb, on=['country_code', 'year'], how='outer', indicator=True)
unmatched = merged[merged['_merge'] != 'both']


### Part c
After this merge, use the `.value_counts()` method to see the total number of observations that were found in both datasets, the number found only in the left dataset, and the number found only in the right dataset. (If you entered the `wb` data frame into the merge function first, then "left_only" refers to the rows found in the World Bank but not V-Dem, and "right_only" refers to the rows found in V-Dem but not the World Bank.) There should be more than 9000 rows that matched, but more than 2000 that failed to match.

Then conduct two data aggregations to help us investigate why these observations did not match:

* First use `.query()` to keep only the observations that were present in `wb` but not `vdem`. (These are the 'left_only' observations if you typed the World Bank data into the merge function first.) Use `.groupby()` to aggregate the data by both 'country_code' and 'country_name_wb'. Then save the minimum and maximum values of 'year' for each country.

* Then use `.query()` to keep only the observations that were present in `vdem` data but not `wb`. Use `.groupby()` to aggregate the data by both 'country_code' and 'country_name_vdem'. Then save the minimum and maximum values of 'year' for each country. [2 points]

In [ ]:
matches = merged['_merge'].value_counts()
left_only = merged[merged['_merge'] == 'left_only'].groupby(['country_code', 'country_name_wb']).agg({'year': ['min', 'max']})
right_only = merged[merged['_merge'] == 'right_only'].groupby(['country_code', 'country_name_vdem']).agg({'year': ['min', 'max']})


### Part d
Here's where a deep understanding of the data becomes very important. There are two reasons why an observation may fail to match in a merge. One reason is a difference in spelling. Suppose that South Korea (which is also known as the Republic of Korea) is coded as SKO in the World Bank data and ROK in V-Dem. In this case, we should recode one or the other of SKO and ROK so that they match, otherwise we will lose the data on South Korea. But the second reason why observations might fail to match is due to differences in coverage in the data collection strategy: it is possible that a country wasn't included in one data's coverage, or that certain years for that country were not included. For differences in coverage, there's no way to manipulate the data to match, so we are out of luck and we have to either delete these observations or proceed with missing data from one of the data sources.

Take a close look at the two data aggregation tables you generated in part (j), and answer the following questions:

* Do you see any countries that are present in both the unmatched World Bank rows and the unmatched V-Dem rows, but with different spellings?

* Do some digging on Wikipedia and other sources on the Internet. What do you think is the primary reason why some countries are present in the V-Dem data but not the World Bank? (You don't need to describe the reasoning for every country. Just dig until you see a general pattern and describe it here.)

* Do some more digging on Wikipedia and other sources on the Internet. What do you think is the primary reason why some countries are present in the World Bank data but not V-Dem? (You don't need to describe the reasoning for every country. Just dig until you see a general pattern and describe it here.) [1 point]   After examining the data, it seems like differences in spelling and naming conventions are the primary reasons for unmatched observations.



### Part e
Once you are convinced that all of the unmatched observations are due to differences in the coverage of the data collection strategies of the World Bank and V-Dem, repeat the merge, dropping all unmatched observations. This time there is no need to validate the type of merge, and no need to define a variable to indicate matching. [1 point]

In [ ]:
merged = merged[merged['_merge'] == 'both']


## Problem 5
Write code using `pandas` that answers the next two questions:

### Part a
Of all countries in the data, which countries have the highest and lowest average levels of democratic quality across the 1960-2022 timespan? [1 point]



In [ ]:
avg_democracy = vdem.groupby('country_name_vdem')['democracy'].mean()
highest_dem_country = avg_democracy.idxmax()
lowest_dem_country = avg_democracy.idxmin()

### Part b
The 'educational_equality' index compiled by V-Dem measures the extent to which "high quality basic education guaranteed to all, sufficient to enable them to exercise their basic rights as adult citizens." They use a Bayesian scaling method to create a score for each country in each year that ranges roughly from -4 to 4, where low values of the scale mean that
> Provision of high quality basic education is extremely unequal and at least 75 percent (%) of children receive such low-quality education that undermines their ability to exercise their basic rights as adult citizens.

And high values mean that
> Basic education is equal in quality and less than five percent (%) of children receive such low-quality education that probably undermines their ability to exercise their basic rights as adult citizens.

Use the `pd.cut()` method to create a categorical version of 'educational_equality' with five categories, one from -4 to -2 called "extremely unequal", one from -2 to -.5 called "very unequal", one from -.5 to .5 called "somewhat unequal", one from .5 to 1.5 called "relatively equal", and one for values from 1.5 to 4 called "equal". (By default, the `pd.cut()` method sets `right=True`, which means the bins include their rightmost edges, so a value of exactly -2 will fall within the "extremely unequal" bin. Leave this default in place.)

Then aggregate the data to have one row per category of the new categorical version of "educational_equality". Collapse the following features to the mean with each category of "educational_equality":

* 'gini_index': The GINI index measures the amount of economic inequality in a country. The higher the index, the greater the economic disparity between rich and poor.
* 'poverty_headcount_ratio_at_national_poverty_lines': a measure of the proportion of the population living in poverty [1 point]
  

In [ ]:
bins = [-4, -2, -0.5, 0.5, 1.5, 4]
labels = ['extremely unequal', 'very unequal', 'somewhat unequal', 'relatively equal', 'equal']
vdem['educational_equality_category'] = pd.cut(vdem['educational_equality'], bins=bins, labels=labels)
result = vdem.groupby('educational_equality_category').agg({'gini_index': 'mean', 'poverty_headcount_ratio_at_national_poverty_lines': 'mean'})
